## Pandas data frame

Data Validation/ingestions

Write pipe separated text file | 

Explore text file

GZIP text file

In [4]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import numpy as np
import datetime 
import gc
import re
import csv

Column validation function

In [5]:
def val_data_col():
    # clean up df columns #
    dfd.columns=dfd.columns.str.replace('[?]', '')
    dfd.columns=dfd.columns.str.strip()
    dfd.columns=dfd.columns.str.replace('[ ]', '_')
    dfd.columns=dfd.columns.str.lower()
    # compare yaml columns with df columns #
    expected_columns = list(file['columns'])
    if len(dfd.columns) == len(expected_columns) and list(dfd.columns) == expected_columns:
        print('column name and column length validation passed')
        mismatched_columns_file = list(set(dfd.columns).difference(expected_columns))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_columns).difference(dfd.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {dfd.columns}')
        logging.info(f'expected columns: {expected_columns}')
        return 1
    else:
        print('column name and column length validation failed')

    return 0

Begin tracing load times and memory use

In [6]:
import pandas as pd
import tracemalloc
tracemalloc.start()
dfd = pd.read_csv('Parking_Violations_2015.csv', delimiter=',')
dfd
snapshot=tracemalloc.take_snapshot()
tracemalloc.stop()

C:\Users\nunto\AppData\Local\Temp\ipykernel_13292\28789275.py:4: DtypeWarning: Columns (18,29,38,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  dfd = pd.read_csv('Parking_Violations_2015.csv', delimiter=',')


In [7]:
top_stats=snapshot.statistics('traceback')
# pick the biggest memory block
stat = top_stats[0]
print("%s memory blocks: %.1f KiB" % (stat.count, stat.size / 1024))
for line in stat.traceback.format():
    print(line)

9 memory blocks: 4705241.6 KiB
  File "C:\Users\nunto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\internals\managers.py", line 2118
    new_values = new_values[argsort]


Open yaml file

In [8]:
with open('file.yaml', 'r') as f:
    file = yaml.safe_load(f)
    file

Drop unwanted columns

In [9]:
dfd=dfd.drop(["BIN", "BBL", "NTA"], axis=1)
dfd=dfd.drop(["Latitude", "Longitude", "Community Board", "Community Council ", "Census Tract"], axis=1)

Validate csv columns

In [10]:
val_data_col()

column name and column length validation passed
Following File columns are not in the YAML file []
Following YAML columns are not in the file uploaded []


C:\Users\nunto\AppData\Local\Temp\ipykernel_13292\630901633.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd.columns=dfd.columns.str.replace('[?]', '')
C:\Users\nunto\AppData\Local\Temp\ipykernel_13292\630901633.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd.columns=dfd.columns.str.replace('[ ]', '_')


1

Create pipe-separated text file

In [ ]:
dfd.to_csv('validated2.txt', sep='|')